In [1]:
import pandas as pd

In [ ]:
# if you are running it on windows
auction_user = pd.read_parquet(r"C:\Users\Chand\Downloads\pranjal_data\usable data\raw_sample_auctions_users.parquet")
auction_results = pd.read_parquet(r"C:\Users\Chand\Downloads\pranjal_data\usable data\raw_sample_auctions_results.parquet")
user_journey  = pd.read_parquet(r"C:\Users\Chand\Downloads\pranjal_data\usable data\user_journey_dataset.parquet")
browsing_session = pd.read_parquet(r"C:\Users\Chand\Downloads\pranjal_data\usable data\browsing_sessions.parquet")
shopping_session = pd.read_parquet(r"C:\Users\Chand\Downloads\pranjal_data\usable data\shopping_sessions.parquet")
product_catalog = pd.read_parquet(r"C:\Users\Chand\Downloads\pranjal_data\usable data\processed_sample_catalog.parquet")
impression = pd.read_parquet(r"C:\Users\Chand\Downloads\pranjal_data\usable data\raw_sample_impressions.parquet")
click = pd.read_parquet(r"C:\Users\Chand\Downloads\pranjal_data\usable data\raw_sample_clicks.parquet")
purchase = pd.read_parquet(r"C:\Users\Chand\Downloads\pranjal_data\usable data\raw_sample_purchases.parquet")
auction_user = auction_user.drop_duplicates()

In [2]:
# if you are running it on mac
auction_user = pd.read_parquet("/Users/mingao/Desktop/data topsort/pranjal_data/usable data/raw_sample_auctions_users.parquet")
auction_results = pd.read_parquet("/Users/mingao/Desktop/data topsort/pranjal_data/usable data/raw_sample_auctions_results.parquet")
user_journey  = pd.read_parquet("/Users/mingao/Desktop/data topsort/pranjal_data/usable data/user_journey_dataset.parquet")
browsing_session = pd.read_parquet("/Users/mingao/Desktop/data topsort/pranjal_data/usable data/browsing_sessions.parquet")
shopping_session = pd.read_parquet("/Users/mingao/Desktop/data topsort/pranjal_data/usable data/shopping_sessions.parquet")
product_catalog = pd.read_parquet("/Users/mingao/Desktop/data topsort/pranjal_data/usable data/processed_sample_catalog.parquet")
impression = pd.read_parquet("/Users/mingao/Desktop/data topsort/pranjal_data/usable data/raw_sample_impressions.parquet")
click = pd.read_parquet("/Users/mingao/Desktop/data topsort/pranjal_data/usable data/raw_sample_clicks.parquet")
purchase = pd.read_parquet("/Users/mingao/Desktop/data topsort/pranjal_data/usable data/raw_sample_purchases.parquet")

auction_user = auction_user.drop_duplicates()

In [3]:
datasets = {
    "auction_user": auction_user,
    "auction_results": auction_results,
    "user_journey": user_journey,
    "browsing_session": browsing_session,
    "shopping_session": shopping_session,
    "product_catalog": product_catalog,
    "impression": impression,
    "click": click,
    "purchase": purchase,
}

for name, df in datasets.items():
    print(f"\n{name} columns:")
    print(df.columns.tolist())



auction_user columns:
['AUCTION_ID', 'OPAQUE_USER_ID', 'CREATED_AT']

auction_results columns:
['AUCTION_ID', 'VENDOR_ID', 'CAMPAIGN_ID', 'PRODUCT_ID', 'RANKING', 'IS_WINNER', 'CREATED_AT']

user_journey columns:
['journey_id', 'auction', 'click', 'impression', 'journey_revenue', 'total_items_purchased', 'did_purchase', 'user_id', 'journey_start_time']

browsing_session columns:
['browsing_session_id', 'user_id', 'shopping_session_id', 'session_start', 'session_end', 'num_events', 'num_auctions', 'num_impressions', 'num_clicks', 'num_purchases', 'session_revenue', 'unique_products', 'unique_auctions', 'duration_minutes', 'session_revenue_usd']

shopping_session columns:
['shopping_session_id', 'user_id', 'shopping_start', 'shopping_end', 'total_events', 'total_auctions', 'total_impressions', 'total_clicks', 'total_purchases', 'total_revenue', 'num_browsing_sessions', 'total_unique_products', 'total_unique_auctions', 'total_browse_minutes', 'shopping_duration_days', 'did_purchase', 'av

In [4]:
print("The auction time is from", auction_results['CREATED_AT'].min(), "to", auction_results['CREATED_AT'].max())
print("The impression time is from ", impression['OCCURRED_AT'].min(), "to", impression['OCCURRED_AT'].max())
print("The purchase time is from ", purchase['PURCHASED_AT'].min(), "to", purchase['PURCHASED_AT'].max())

# find the list of user_id that appears in auctions, note that they may not appear in the impression data
user_ids  = auction_user['OPAQUE_USER_ID'].unique().tolist()

The auction time is from 2025-03-25 00:00:27.680000 to 2025-09-20 23:59:31.906000
The impression time is from  2025-03-25 00:01:05 to 2025-09-20 23:59:56
The purchase time is from  2025-03-25 01:18:17 to 2025-09-20 23:41:05


In [5]:
# Get unique IDs from both datasets
results_ids = set(auction_results["PRODUCT_ID"].unique())
catalog_ids = set(product_catalog["PRODUCT_ID"].unique())

# Find overlap
overlap_ids = results_ids & catalog_ids

print("Number of unique PRODUCT_IDs in auction_results:", len(results_ids))
print("Number of unique PRODUCT_IDs in product_catalog:", len(catalog_ids))
print("Number of overlapping unique PRODUCT_IDs:", len(overlap_ids))

Number of unique PRODUCT_IDs in auction_results: 3980626
Number of unique PRODUCT_IDs in product_catalog: 3981005
Number of overlapping unique PRODUCT_IDs: 3980421


In [ ]:
data = {
    'auction_user': auction_user,
    'auction_results': auction_results,
    'impression': impression,
    'click': click,
    'purchase': purchase
}

df_user = pd.DataFrame(data['auction_user'])
df_results = pd.DataFrame(data['auction_results'])
df_impression = pd.DataFrame(data['impression'])
df_click = pd.DataFrame(data['click'])
df_purchase = pd.DataFrame(data['purchase'])

# --- 2. Data Type Conversion (Time Data) ---
time_cols = ['CREATED_AT']
for df, cols in zip(
    [df_results, df_impression, df_click, df_purchase],
    [time_cols, time_cols, time_cols, ['PURCHASED_AT']]
):
    for col in cols:
        # Check if the column exists and convert
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')


# --- 3. Initial Base Join: auction_results + auction_user ---
# This ensures every auction result is linked to its OPAQUE_USER_ID if available.
df_base = df_results.merge(
    df_user[['AUCTION_ID', 'OPAQUE_USER_ID']],
    on='AUCTION_ID',
    how='left'
)

df_base = df_base.rename(columns={'CREATED_AT': 'AUCTION_TIME'})


# --- 4. Funnel Step 1: Base -> Impression (Left Join: Keep all auction results) ---
# Key: AUCTION_ID, PRODUCT_ID
df_funnel_I = df_base.merge(
    df_impression.add_prefix('IMPR_'),
    left_on=['AUCTION_ID', 'PRODUCT_ID'],
    right_on=['IMPR_AUCTION_ID', 'IMPR_PRODUCT_ID'],
    how='left',
    suffixes=('_AUCTION', '_IMPR')
)
# This is the step that introduces the crucial lowercase USER_ID
df_funnel_I.rename(columns={'IMPR_USER_ID': 'USER_ID'}, inplace=True)
df_funnel_I.drop(columns=['IMPR_AUCTION_ID', 'IMPR_PRODUCT_ID'], inplace=True)


# --- 5. Funnel Step 2: Impression -> Click (Left Join) ---
# Key: AUCTION_ID, PRODUCT_ID, USER_ID (The user who saw the impression must be the one who clicked)
df_funnel_IC = df_funnel_I.merge(
    df_click.add_prefix('CLICK_'),
    left_on=['AUCTION_ID', 'PRODUCT_ID', 'USER_ID'],
    right_on=['CLICK_AUCTION_ID', 'CLICK_PRODUCT_ID', 'CLICK_USER_ID'],
    how='left',
    suffixes=('_IMPR', '_CLICK')
)
df_funnel_IC.drop(columns=[
    'CLICK_AUCTION_ID', 'CLICK_PRODUCT_ID', 'CLICK_USER_ID'
], inplace=True)


# --- 6. Funnel Step 3: Click -> Purchase (Left Join with Time Constraint) ---

# Initial left merge on common keys: PRODUCT_ID and USER_ID
df_funnel_ICP = df_funnel_IC.merge(
    df_purchase.add_prefix('PURCH_'),
    left_on=['PRODUCT_ID', 'USER_ID'],
    right_on=['PURCH_PRODUCT_ID', 'PURCH_USER_ID'],
    how='left'
)
df_funnel_ICP.drop(columns=['PURCH_PRODUCT_ID', 'PURCH_USER_ID'], inplace=True)

# Apply the time constraint filter (Purchase must be 0 < time_diff <= 180 minutes after Click)
time_diff = df_funnel_ICP['PURCH_PURCHASED_AT'] - df_funnel_ICP['CLICK_OCCURRED_AT']
time_window = pd.Timedelta(minutes=180)

# Identify rows where a purchase occurred (not NaN) AND the time constraint is NOT met
invalid_purchase_mask = (
    df_funnel_ICP['PURCH_PURCHASED_AT'].notna() &
    (
        # The click time must not be NaN for the comparison
        df_funnel_ICP['CLICK_OCCURRED_AT'].isna() |
        # Purchase must be AFTER the click (time_diff > 0) OR within the window (time_diff <= 180 min)
        (time_diff < pd.Timedelta(seconds=0)) | 
        (time_diff > time_window)
    )
)

# Nullify the purchase-related columns for these invalid rows
purchase_cols = [col for col in df_funnel_ICP.columns if col.startswith('PURCH_')]
df_funnel_ICP.loc[invalid_purchase_mask, purchase_cols] = pd.NA

# Final cleanup and column selection
df_funnel_ICP.rename(
    columns={
        'PURCH_PURCHASED_AT': 'PURCHASE_TIME',
        'PURCH_PURCHASE_ID': 'PURCHASE_ID',
        'PURCH_QUANTITY': 'PURCHASE_QUANTITY',
        'PURCH_UNIT_PRICE': 'PURCHASE_UNIT_PRICE'
    },
    inplace=True
)

# --- 7. Final Funnel Output ---
funnel_cols = [
    'AUCTION_ID', 'AUCTION_TIME', 'PRODUCT_ID', 'VENDOR_ID', 'RANKING', 'IS_WINNER',
    'OPAQUE_USER_ID',
    # 'USER_ID',  # you’re dropping this anyway
    'IMPR_INTERACTION_ID', 'IMPR_OCCURRED_AT',
    'CLICK_INTERACTION_ID', 'CLICK_OCCURRED_AT',
    'PURCHASE_ID', 'PURCHASE_TIME', 'PURCHASE_QUANTITY', 'PURCHASE_UNIT_PRICE'
]

df_funnel = df_funnel_ICP[funnel_cols]


print("--- Final Auction Funnel (User-Augmented Base) ---")
print(f"\nTotal Funnel Records: {len(df_funnel)}")

In [7]:
df_funnel.columns

Index(['AUCTION_ID', 'AUCTION_TIME', 'PRODUCT_ID', 'VENDOR_ID', 'RANKING',
       'IS_WINNER', 'OPAQUE_USER_ID', 'IMPR_INTERACTION_ID',
       'IMPR_OCCURRED_AT', 'CLICK_INTERACTION_ID', 'CLICK_OCCURRED_AT',
       'PURCHASE_ID', 'PURCHASE_TIME', 'PURCHASE_QUANTITY',
       'PURCHASE_UNIT_PRICE'],
      dtype='object')

In [65]:
import pandas as pd

def create_user_timeline(df_funnel, df_product_catalog):
    """
    Transforms the funnel dataset into a user-centric timeline of events
    (impression, click, purchase).

    Args:
        df_funnel (pd.DataFrame): The input DataFrame with funnel data.

    Returns:
        pd.DataFrame: A long-format DataFrame representing the user timeline.
    """

    # --- 1. Impression Events ---
    # Create a DataFrame for Impression events
    df_impr = df_funnel.dropna(subset=['IMPR_INTERACTION_ID']).copy()
    df_impr['EVENT_TYPE'] = 'impression'
    df_impr['EVENT_ID'] = df_impr['IMPR_INTERACTION_ID']
    df_impr['EVENT_TIME'] = df_impr['IMPR_OCCURRED_AT']
    df_impr['EVENT_PRODUCT_ID'] = df_impr['PRODUCT_ID']
    # Purchase-related columns are NaN for impression events
    df_impr['PURCHASE_QUANTITY'] = pd.NA
    df_impr['PURCHASE_UNIT_PRICE'] = pd.NA

    # Select and rename columns for the final timeline structure
    df_impr = df_impr[[
        'OPAQUE_USER_ID', 'AUCTION_ID', 'RANKING', 'IS_WINNER', 'AUCTION_TIME',
        'EVENT_PRODUCT_ID', 'EVENT_ID', 'EVENT_TIME', 'EVENT_TYPE',
        'PURCHASE_QUANTITY', 'PURCHASE_UNIT_PRICE'
    ]]

    # --- 2. Click Events ---
    # Create a DataFrame for Click events
    df_click = df_funnel.dropna(subset=['CLICK_INTERACTION_ID']).copy()
    df_click['EVENT_TYPE'] = 'click'
    df_click['EVENT_ID'] = df_click['CLICK_INTERACTION_ID']
    df_click['EVENT_TIME'] = df_click['CLICK_OCCURRED_AT']
    df_click['EVENT_PRODUCT_ID'] = df_click['PRODUCT_ID']
    # Purchase-related columns are NaN for click events
    df_click['PURCHASE_QUANTITY'] = pd.NA
    df_click['PURCHASE_UNIT_PRICE'] = pd.NA

    # Select and rename columns for the final timeline structure
    df_click = df_click[[
        'OPAQUE_USER_ID', 'AUCTION_ID', 'RANKING', 'IS_WINNER', 'AUCTION_TIME',
        'EVENT_PRODUCT_ID', 'EVENT_ID', 'EVENT_TIME', 'EVENT_TYPE',
        'PURCHASE_QUANTITY', 'PURCHASE_UNIT_PRICE'
    ]]

    # --- 3. Purchase Events ---
    # Create a DataFrame for Purchase events
    # A purchase is typically tied to the product/auction that led to it,
    # so we use the existing columns.
    df_purchase = df_funnel.dropna(subset=['PURCHASE_ID']).copy()
    df_purchase['EVENT_TYPE'] = 'purchase'
    df_purchase['EVENT_ID'] = df_purchase['PURCHASE_ID']
    df_purchase['EVENT_TIME'] = df_purchase['PURCHASE_TIME']
    df_purchase['EVENT_PRODUCT_ID'] = df_purchase['PRODUCT_ID']
    # Retain the actual purchase quantity/price
    # Note: These are already present in the source df_funnel, we just need to select them.

    # Select and rename columns for the final timeline structure
    df_purchase = df_purchase[[
        'OPAQUE_USER_ID', 'AUCTION_ID', 'RANKING', 'IS_WINNER', 'AUCTION_TIME',
        'EVENT_PRODUCT_ID', 'EVENT_ID', 'EVENT_TIME', 'EVENT_TYPE',
        'PURCHASE_QUANTITY', 'PURCHASE_UNIT_PRICE'
    ]]

    # --- 4. Combine and Sort ---
    # Combine the three event DataFrames
    df_timeline = pd.concat([df_impr, df_click, df_purchase], ignore_index=True)

    # Convert time columns to datetime objects if they aren't already
    # This is crucial for correct sorting.
    time_cols = ['AUCTION_TIME', 'EVENT_TIME']
    for col in time_cols:
        if df_timeline[col].dtype != 'datetime64[ns]':
            df_timeline[col] = pd.to_datetime(df_timeline[col], errors='coerce')


    # Sort the combined timeline:
    # 1. By OPAQUE_USER_ID
    # 2. Then by AUCTION_TIME (time of the auction where the event occurred)
    # 3. Finally, by EVENT_TIME (time of the specific event: impr, click, or purchase)
    df_timeline = df_timeline.sort_values(
        by=['OPAQUE_USER_ID', 'AUCTION_TIME', 'EVENT_TIME']
    ).reset_index(drop=True)

    catalog_cols = ['PRODUCT_ID', 'NAME', 'PRICE', 'BRAND', 'PRIMARY_COLOR']
    df_catalog_subset = df_product_catalog[catalog_cols].copy()

    # Perform a LEFT MERGE to add product details to the timeline.
    # We use a left merge to ensure all timeline events are kept, even if a
    # product is missing from the catalog (which would result in NaNs for the
    # catalog columns).
    df_timeline_enriched = pd.merge(
        df_timeline,
        df_catalog_subset,
        how='left',
        left_on='EVENT_PRODUCT_ID',
        right_on='PRODUCT_ID'
    )


    # Drop the redundant 'PRODUCT_ID' column from the merge
    df_timeline_enriched = df_timeline_enriched.drop(columns=['PRODUCT_ID'])

    return df_timeline_enriched

In [ ]:
df_timeline_result = create_user_timeline(df_funnel, product_catalog)

In [69]:
df_timeline_result.columns

Index(['OPAQUE_USER_ID', 'AUCTION_ID', 'RANKING', 'IS_WINNER', 'AUCTION_TIME',
       'EVENT_PRODUCT_ID', 'EVENT_ID', 'EVENT_TIME', 'EVENT_TYPE',
       'PURCHASE_QUANTITY', 'PURCHASE_UNIT_PRICE', 'NAME', 'PRICE', 'BRAND',
       'PRIMARY_COLOR'],
      dtype='object')

In [70]:
import pandas as pd

def get_user_pre_purchase_timeline(df_timeline):
    """
    Selects a random user who made a purchase and returns their activity
    in the hour leading up to their first purchase.

    Args:
        df_timeline (pd.DataFrame): The enriched user timeline DataFrame.

    Returns:
        pd.DataFrame: A filtered DataFrame of the selected user's activity
                      one hour before their purchase.
        None: If no user with a purchase is found.
    """
    # Ensure EVENT_TIME is in datetime format for reliable time calculations
    if df_timeline['EVENT_TIME'].dtype != 'datetime64[ns]':
        df_timeline['EVENT_TIME'] = pd.to_datetime(df_timeline['EVENT_TIME'], errors='coerce')

    # --- 1. Identify users who made a purchase ---
    purchase_users = df_timeline[df_timeline['EVENT_TYPE'] == 'purchase']['OPAQUE_USER_ID'].unique()

    if len(purchase_users) == 0:
        print("No purchase events found in the timeline.")
        return None

    # --- 2. Select a random user and their first purchase time ---
    selected_user_id = pd.Series(purchase_users).sample(n=1).iloc[0]

    # Filter for the selected user's purchases
    user_purchases = df_timeline[
        (df_timeline['OPAQUE_USER_ID'] == selected_user_id) &
        (df_timeline['EVENT_TYPE'] == 'purchase')
    ].sort_values(by='EVENT_TIME')

    # Get the time of their first purchase
    first_purchase_time = user_purchases['EVENT_TIME'].iloc[0]

    # --- 3. Define the time window (1 hour before purchase) ---
    start_time_window = first_purchase_time - pd.Timedelta(hours=1)
    end_time_window = first_purchase_time # The window includes events *up to* the purchase

    # --- 4. Filter the timeline for the user and the time window ---
    user_timeline_activity = df_timeline[
        (df_timeline['OPAQUE_USER_ID'] == selected_user_id) &
        (df_timeline['EVENT_TIME'] >= start_time_window) &
        (df_timeline['EVENT_TIME'] <= end_time_window)
    ].sort_values(by='EVENT_TIME')

    print(f"--- Activity for User ID: {selected_user_id} ---")
    print(f"First Purchase Time: {first_purchase_time}")
    print(f"Viewing window: {start_time_window} to {end_time_window}")
    print("-" * 40)

    return user_timeline_activity

# --- Example Usage (assuming df_timeline_result is the DataFrame) ---
# pre_purchase_activity = get_user_pre_purchase_timeline(df_timeline_result)

# if pre_purchase_activity is not None:
#     print(pre_purchase_activity)

In [71]:
pre_purchase_activity = get_user_pre_purchase_timeline(df_timeline_result)

if pre_purchase_activity is not None:
     print(pre_purchase_activity)

--- Activity for User ID: ext1:50c7ba00-1b7d-4bcc-bb00-164819555116 ---
First Purchase Time: 2025-05-23 20:03:26
Viewing window: 2025-05-23 19:03:26 to 2025-05-23 20:03:26
----------------------------------------
                                   OPAQUE_USER_ID  \
322782  ext1:50c7ba00-1b7d-4bcc-bb00-164819555116   
322783  ext1:50c7ba00-1b7d-4bcc-bb00-164819555116   
322784  ext1:50c7ba00-1b7d-4bcc-bb00-164819555116   
322785  ext1:50c7ba00-1b7d-4bcc-bb00-164819555116   
322786  ext1:50c7ba00-1b7d-4bcc-bb00-164819555116   
322787  ext1:50c7ba00-1b7d-4bcc-bb00-164819555116   
322788  ext1:50c7ba00-1b7d-4bcc-bb00-164819555116   
322789  ext1:50c7ba00-1b7d-4bcc-bb00-164819555116   
322790  ext1:50c7ba00-1b7d-4bcc-bb00-164819555116   
322791  ext1:50c7ba00-1b7d-4bcc-bb00-164819555116   
322792  ext1:50c7ba00-1b7d-4bcc-bb00-164819555116   
322793  ext1:50c7ba00-1b7d-4bcc-bb00-164819555116   
322795  ext1:50c7ba00-1b7d-4bcc-bb00-164819555116   
322794  ext1:50c7ba00-1b7d-4bcc-bb00-16481955